## Latest Process code

Below the process function definition to upload ALL the lat-lon pairs of the data.
Running the process function over one file (e.g. the file in here) takes 3.5h which quite a lot for just one file.
The main reason seems to be the object creation of the "c3.SurfaceHindcastData" takes 0.3s per object. This seems strange as it should be just an oject initialization, why does it take so long?

Would be awesome if you can investigate that @Darren and maybe you have ideas how to speed that up?

In [7]:
process(c3.HindcastFile.get('007e88f7-b86d-4379-a194-a73e2d003afb'))

In [14]:
from datetime import datetime, timedelta
def gentimes(start,end,stride):
        t = start
        while t <= end:
            yield t
            t += timedelta(hours=stride)

In [42]:
g = gentimes(datetime(2021,1,1),datetime(2021,1,2),1)

In [45]:
for t in g:
    print(t)

2021-01-01 01:00:00
2021-01-01 02:00:00
2021-01-01 03:00:00
2021-01-01 04:00:00
2021-01-01 05:00:00
2021-01-01 06:00:00
2021-01-01 07:00:00
2021-01-01 08:00:00
2021-01-01 09:00:00
2021-01-01 10:00:00
2021-01-01 11:00:00
2021-01-01 12:00:00
2021-01-01 13:00:00
2021-01-01 14:00:00
2021-01-01 15:00:00
2021-01-01 16:00:00
2021-01-01 17:00:00
2021-01-01 18:00:00
2021-01-01 19:00:00
2021-01-01 20:00:00
2021-01-01 21:00:00
2021-01-01 22:00:00
2021-01-01 23:00:00
2021-01-02 00:00:00


In [49]:
latitudes = range(2)
longitudes = range(2)
genRecords = (
        c3.SurfaceHindcastData( # look at the code to understand, TimeDataPoint (Check out Type)
            **{
                'start': time,
                'parent': 'HNDCST_SRFC_' + str(i) + '-' + str(j),
                'name': 'water_u',  # variable
                'for': time,           # timestamp
                'water_u': hycom_file.variables['water_u'][:].data[0,0,i,j],
                'water_v': hycom_file.variables['water_v'][:].data[0,0,i,j]
            }
        )
        for i in latitudes
            for j in longitudes
)
       

In [50]:
next(genRecords)

NameError: name 'hycom_file' is not defined

In [1]:
from datetime import datetime, timedelta
def processSync(this):
    """ Process a single Hindcast NetCDF file into the Hindcast__Data types"""
    # extract surface data for a variable
    hycom_file = c3.HycomUtil.nc_open(this.file.url)
    
    # extract lat-long, or derive this from types
    # Not yet done:determine the offset for each based on the subsetOptions for this file
    # Note: for now it's just an integer list
    latitudes = range(len(hycom_file['lat']))
    longitudes = range(len(hycom_file['lon']))
    
    def gentimes(start,end,stride):
        t = start
        while t <= end:
            yield t
            t += timedelta(hours=stride)
    
    times = list(gentimes(this.start,this.end,this.subsetOptions.timeStride))

    # Create list of instantiated SurfaceHindcastData types
    # Create a parent id for each lat-long pair in the file.
    # use lat-long indicies to create a string parent id
    it = 0
    for time in times:
        print(f"Reading data arrays for time: {time}")
        water_u = hycom_file.variables['water_u'][:].data[it,0,:,:]
        water_v = hycom_file.variables['water_v'][:].data[it,0,:,:]
        print("Creating records...")
        data_records = [
#             c3.SurfaceHindcastData( 
#                 **{
#                     'start': time,
#                     'parent': 'HNDCST_SRFC_' + str(i) + '-' + str(j),
#                     'name': 'water_u',  # variable
#                     'for': time,           # timestamp
#                     'water_u': water_u[i,j],
#                     'water_v': water_v[i,j]
#                 }
            c3.SurfaceHindcastData( 
                start = time,
                parent = 'HNDCST_SRFC_' + str(i) + '-' + str(j),
                name = 'water_u',
                water_u = water_u[i,j],
                water_v = water_v[i,j]
            )
            for i in latitudes
                for j in longitudes
        ]
        print(f"upserting {len(data_records)}")
        it += 1
        #upsert to data store
        #c3.SurfaceHindcastData.upsertBatch(data_records)
        break
    

    # close the file ds, url
    c3.HycomUtil.nc_close(ds=hycom_file, url=this.file.url)
    return actions
    # aggregation in space is possible but a bit more complex

In [2]:
file = c3.HindcastFile.get('55c392d8-9d83-478f-85b0-f1c0037ef4ee/GOMu0.04-expt_90.1m000-2021-2021-09-01T12:00:00Z-2021-09-02T11:00:00Z.nc')
actions = processSync(file)

Reading data arrays for time: 2021-09-01 12:00:00
Creating records...


KeyboardInterrupt: 

In [75]:
# Prototype for prosessing hindcast files
from datetime import datetime, timedelta
from itertools import islice

def chunk(gen, k):
    while True:
        chunk = [*islice(gen, 0, k)]
        if chunk:
            yield chunk
        else:
            break

def process(this,chunkSize=5000):
    """ Process a single Hindcast NetCDF file into the Hindcast__Data types"""
    # extract surface data for a variable
    hycom_file = c3.HycomUtil.nc_open(this.file.url)
    
    # extract lat-long, or derive this from types
    # Not yet done:determine the offset for each based on the subsetOptions for this file
    # Note: for now it's just an integer list
    xsz = len(hycom_file['lon'])
    ysz = len(hycom_file['lat'])
    latitudes = range(ysz)
    longitudes = range(xsz)
    
    # Generate a list of times that the file contains
    def gentimes(start,end,stride):
        t = start
        while t <= end:
            yield t
            t += timedelta(hours=stride)
    
    times = list(gentimes(this.start,this.end,this.subsetOptions.timeStride))
    print (f"Processing {len(times)}")

    # Loop over timesteps
    # Use a generator to instatiate types in batches
    actions = []
    it = 0
    def idx(i,j):
        return ysz*i  + j
    for time in times:
        print(f"time: {time}")
        water_u = hycom_file.variables['water_u'][:].data[it,0,:,:]
        water_v = hycom_file.variables['water_v'][:].data[it,0,:,:]
        print(water_u.shape)
        genRecords = (
            c3.SurfaceHindcastData( # look at the code to understand, TimeDataPoint (Check out Type)
                **{
                    'start': time,
                    'parent' : c3.SurfaceHindcastDataSeries(
                        id = 'HNDCST_SRFC_' + str(i) + '-' + str(j)
                    ).toJson(),
                    'name': 'water_u',
                    'water_u': water_u[i,j],
                    'water_v': water_v[i,j]
                }
            )
            for i in latitudes
                for j in longitudes
        )
        
#         print(f"generating {chunkSize} records...")
#         objs = next(chunk(genRecords,chunkSize))
#         print("done")
        
        # Asychronosly submit upserts 
        ic = 1
        for objs in chunk(genRecords,chunkSize):
            print (f"chunk: {ic}")
            action = c3.AsyncAction.submit({
                'typeName': "SurfaceHindcastData",
                'action': 'upsertBatch',
                'args': {
                    'objs': objs
                 }
            })
            actions.append(action)
            ic += 1
        it += 1
        break
    # upsert to data store
    #c3.SurfaceHindcastData.upsertBatch(data_records)

    # close the file
    c3.HycomUtil.nc_close(ds=hycom_file, url=this.file.url)
    return actions
    

In [ ]:
file = c3.HindcastFile.get('55c392d8-9d83-478f-85b0-f1c0037ef4ee/GOMu0.04-expt_90.1m000-2021-2021-09-01T12:00:00Z-2021-09-02T11:00:00Z.nc')

In [76]:
objs = process(file,10000)

time: 2021-09-01 12:00:00
(346, 541)
generating 10000 records...
done


# DETOUR

In [ ]:
# Prototype for prosessing hindcast files
from datetime import datetime, timedelta
from itertools import islice
import pandas as pd

def chunk(gen, k):
    while True:
        chunk = [*islice(gen, 0, k)]
        if chunk:
            yield chunk
        else:
            break

def process(this,chunkSize=5000):
    """ Process a single Hindcast NetCDF file into the Hindcast__Data types"""
    # extract surface data for a variable
    hycom_file = c3.HycomUtil.nc_open(this.file.url)
    
    # extract lat-long, or derive this from types
    # Not yet done:determine the offset for each based on the subsetOptions for this file
    # Note: for now it's just an integer list
    xsz = len(hycom_file['lon'])
    ysz = len(hycom_file['lat'])
    latitudes = range(ysz)
    longitudes = range(xsz)
    
    # Generate a list of times that the file contains
    def gentimes(start,end,stride):
        t = start
        while t <= end:
            yield t
            t += timedelta(hours=stride)
    
    times = list(gentimes(this.start,this.end,this.subsetOptions.timeStride))
    print (f"Processing {len(times)}")

    # Loop over timesteps
    # Use a generator to instatiate types in batches
    actions = []
    it = 0
    def idx(i,j):
        return ysz*i  + j
    
    
    lats = # something N^2 list
    lons = # something N^2 list
    parents = N*N*['HNDCST_SRFC_'] + lats + N*N*['-'] + lons
    for time in times:
        print(f"time: {time}")
        df = pd.DataFrame()
        df['water_u'] = hycom_file.variables['water_u'][:].data[it,0,:,:]
        df['water_v'] = hycom_file.variables['water_v'][:].data[it,0,:,:]
        df['name'] = 'water_u'
        df['start'] = time
        df['parent'] = parents
        recs = df.to_dict(orient="records")
        c3.SurfaceHindcastData.upsertBatch(recs)

    

In [78]:
objs[0]

c3.SurfaceHindcastData(
 parent=c3.SurfaceHindcastDataSeries(id='HNDCST_SRFC_0-0'),
 start=datetime.datetime(2021, 9, 1, 12, 0),
 name='water_u',
 water_u=-30000.0,
 water_v=-30000.0)

In [ ]:
i=range(2)
j=range(3)
ij = 

## Data Loading

In [1]:
import datetime
# Example: Define archive for surface `water_u` and `water_v` on a lat-long subset for the month of September 2021
hcast = c3.HycomHindcast.get("GOMu0.04-expt_90.1m000-2021")
archv = c3.HindcastArchive(
    **{
        "hindcast": hcast,
        "description": "Test subset with vertCoord=4 ",
        "subsetOptions": c3.HycomSubsetOptions(
            **{
                "timeRange": {
                    "start": datetime.datetime(2021,9,1,12),
                    "end": datetime.datetime(2021,9,11,11)
                },
                "vars": "water_u,water_v",
                "vertCoord": 4
            }
        ).toJson(),
        "downloadOptions": c3.HycomDownloadOptions(
            **{
                'externalDir': 'hycom-test',
                'maxTimesPerFile': 24
            }
        ).toJson()
        
    }
)
archv.upsert()
#archv

c3.HindcastArchive(
 id='55c392d8-9d83-478f-85b0-f1c0037ef4ee',
 meta=c3.Meta(
        tenantTagId=150,
        tenant='dev',
        tag='tc01d',
        created=datetime.datetime(2021, 10, 26, 13, 34, 5, tzinfo=datetime.timezone.utc),
        createdBy='dadams@illinois.edu',
        updated=datetime.datetime(2021, 10, 26, 13, 34, 5, tzinfo=datetime.timezone.utc),
        updatedBy='dadams@illinois.edu',
        timestamp=datetime.datetime(2021, 10, 26, 13, 34, 5, tzinfo=datetime.timezone.utc),
        fetchInclude='[this,{hindcast:[name,id]}]',
        fetchType='HindcastArchive'),
 version=65537,
 hindcast=c3.HycomHindcast(
            id='GOMu0.04-expt_90.1m000-2021',
            name='GOMu0.04-expt_90.1m000-2021'),
 description='Test subset with vertCoord=4 ',
 subsetOptions=c3.HycomSubsetOptions(
                 timeRange=c3.TimeRange(
                             start=datetime.datetime(2021, 9, 1, 12, 0),
                             end=datetime.datetime(2021, 9, 11, 11, 0)),

In [2]:
# Create Batch job
job = c3.HindcastDownloadJob(
    **{
        'options': c3.HycomDownloadJobOptions(
            **{
                'limit': 300
            }
        ).toJson()
    }
).upsert()
job.start()

c3.BatchJobStatus(
 started=datetime.datetime(2021, 10, 26, 13, 34, 55, tzinfo=datetime.timezone.utc),
 startedby='dadams@illinois.edu',
 status='running')

In [3]:
import time
from IPython.display import clear_output
status = job.status()
while status.status == 'running':
    archv = c3.HindcastArchive.get("55c392d8-9d83-478f-85b0-f1c0037ef4ee")
    clear_output()
    status = job.status()
    #gom_dataset = c3.HycomDataset.fetch(spec={'include':"this,hindcastArchiveSize"}).objs[0]
    #print (gom_dataset)
    print(f"Archive Size: {round(archv.archiveSize/(1024**3),5)} GiB")
    filecount = c3.HindcastFile.fetchCount(spec={'filter':"hindcastArchive.id=='"+archv.id+"' && status=='downloaded'"})
    allcount = c3.HindcastFile.fetchCount(spec={'filter':"hindcastArchive.id=='"+archv.id+"'"})
    print(f"Download count: {filecount} of {allcount}")
    print (status)
    time.sleep(20)

Archive Size: 0.0709 GiB
Download count: 10 of 10
c3.BatchJobStatus(
 started=datetime.datetime(2021, 10, 26, 13, 34, 55, tzinfo=datetime.timezone.utc),
 startedby='dadams@illinois.edu',
 completed=datetime.datetime(2021, 10, 26, 13, 37, 43, tzinfo=datetime.timezone.utc),
 status='completed',
 newBatchSubmitted=False)


In [6]:
job.status()

c3.BatchJobStatus(
 started=datetime.datetime(2021, 10, 26, 12, 59, 40, tzinfo=datetime.timezone.utc),
 startedby='dadams@illinois.edu',
 completed=datetime.datetime(2021, 10, 26, 13, 0, 8, tzinfo=datetime.timezone.utc),
 status='failed',
 errors=c3.Arry<JobRunErrorDetail>([c3.JobRunErrorDetail(
           failedActionId='3184.123123157',
           errorMsg='Error executing command: '
                     '/usr/local/share/c3/condaEnvs/dev/tc01d/py-hycom_1_0_0/bin/python '
                     '/tmp/pythonActionSourceCache5863133234282574632/HindcastFile_download.py\n'
                     'p_logger=main url=http://dev-dti-app-w-002:8080 '
                     'connector=null mode="thick" Action failed!\n'
                     "AttributeError: 'HycomSubsetOptions' object has no "
                     "attribute 'vertCord'\n"
                     '\n'
                     'The above exception was the direct cause of the '
                     'following exception:\n'
                  